# ０　元ネタ：ここを参考に実装しました
 - アプリ開発：Jupyter上でDashを使えるjupyter_dash
  - https://qiita.com/OgawaHideyuki/items/725f4ffd93ffb0d30b6c

# １　必要なライブラリのインストール

In [ ]:
!pip install jupyter_dash dash-bootstrap-components
!pip3 install mecab-python3 unidic-lite
!pip3 install wordcloud
!apt-get -y install fonts-ipafont-gothic

# ２　ライブラリのインポート

In [ ]:
from wordcloud import WordCloud
import sys
sys.path.append('//content/drive/MyDrive/0_HanPy41/1_Webアプリケーション') # 実行前にマウントすること！
import connpass_api
from jupyter_dash import JupyterDash 
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from io import BytesIO
import base64
import sqlite3
from contextlib import closing
import datetime
import pytz

PATH = "/content/drive/MyDrive/0_HanPy41/1_Webアプリケーション/words.db"
fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
COUNT = 20

# ３　ＲＤＢの作成
- ＤＢ名：words.db
- テーブル名：master
- 列名
 - name text primary key　（connpassのニックネーム）
 - words text　（名詞をスペース区切りした文字列）
 - date text　（ＲＤＢに登録した日付）

In [ ]:
# import sqlite3

# PATH = "/content/drive/MyDrive/0_HanPy41/1_Webアプリケーション/words.db"
# conn = sqlite3.connect(PATH)
# c = conn.cursor()

# c.execute('CREATE TABLE master (name text primary key, words text, date text)')

# conn.commit()
# conn.close()

# ３　ワードクラウドＷｅｂアプリ（レスポンス改善）の実装

In [ ]:
fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'
COUNT = 20

app = JupyterDash(__name__,
                external_stylesheets=[dbc.themes.SUPERHERO],
                # external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{
                    'name': 'viewport',
                    'content': 'width=device-width, initial-scale=1.0'
                    }]
                )

# View
title = html.Div([
    dbc.Row([
        dbc.Col(
            dbc.Alert(html.H3("Connpass Word Cloud"), 
                      color="primary", style={'text-align':'center'}
                      )
        ),
    ], justify="center")
])

input_name = html.Div([
    dbc.Row([
        dbc.Col(
            html.H4('nick name：'), width="auto"
        ),
        dbc.Col(
            dcc.Input(
                id = 'id_name',
                placeholder='Press Enter to apply',
                type='text',
                value='',
                debounce=True,
                autoFocus=True,
            ), width="auto"
        ),
    ], no_gutters=True, justify="center")
])

output_wc = html.Div([
    dbc.Row([
        dbc.Col(
            dcc.Loading(id="id_loading",
                children=[
                        html.Img(id="id_wc"),
                ],
                type="default",
            ), width="auto"
        ),
    ], justify="center")
], style={'margin':'10px 0px 0px 0px'})

app.layout = dbc.Container([
                            title, 
                            input_name, 
                            output_wc,
                            ])

# Controller
@app.callback(
	Output('id_wc', 'src'),
	[Input('id_name', 'value')],
)
def make_image(nick_name):
    if nick_name == '':
        words = 'ニックネームを入力して下さい'
    else:
        conn = sqlite3.connect(PATH)
        c = conn.cursor()
        row = c.execute(f"SELECT * FROM master WHERE name = '{nick_name}'")
        data = row.fetchall() # nick_name 有：長さ１のリスト　無：長さ０のリスト
        if data:
            words = data[0][1]
        else:
            words = connpass_api.get_words(nick_name, COUNT)
            if words != '該当ありません':
                now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
                date = f'{now.year}/{now.month}/{now.day}'
                SQL = f"INSERT INTO master (name, words, date) VALUES ('{nick_name}', '{words}', '{date}')"
                c.execute(SQL)
        conn.commit()
        conn.close()

    wc = WordCloud(width=400, height=400, background_color='white', font_path=fpath)
    wc.generate(words)
    img = BytesIO()
    wc.to_image().save(img, format='PNG')
    return f'data:image/png;base64,{base64.b64encode(img.getvalue()).decode()}'

# ４　Ｗｅｂアプリをローカル環境で実行

In [ ]:
app.run_server(mode="inline")

# ５　ＲＤＢの登録内容の確認

In [ ]:
import sqlite3
import pandas as pd

PATH = "/content/drive/MyDrive/0_HanPy41/1_Webアプリケーション/words.db"

conn = sqlite3.connect(PATH)
SQL = f"SELECT * FROM master"
df = pd.read_sql(SQL, conn)

df

# ６　Ｗｅｂアプリのデプロイ

## externalモードでWebアプリを起動

In [ ]:
app.run_server(mode='external', host="localhost", port=8050, debug=False)

## トンネリングで公開ＵＲＬを発行

In [ ]:
# cloudflaredのインストール　＆　localhostの8050ポートのトンネリングした公開URLを発行
!wget https://bin.equinox.io/c/VdrWdbjqyF/cloudflared-stable-linux-amd64.deb
!dpkg -i cloudflared-stable-linux-amd64.deb
!cloudflared tunnel --url localhost:8050